In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging

import pandas as pd
import torch
import torch.nn.functional as F
from rae.modules.enumerations import Output
from rae.pl_modules.pl_gautoencoder import LightningAutoencoder

try:
    # be ready for 3.10 when it drops
    from enum import StrEnum
except ImportError:
    from backports.strenum import StrEnum

import matplotlib.pyplot as plt
from tueplots import bundles
from tueplots import figsizes

logging.getLogger().setLevel(logging.ERROR)

DEVICE: str = "cpu"

In [ ]:
from rae.modules.text.encoder import GensimEncoder

ENCODERS = {
    model_name: GensimEncoder(language="en", lemmatize=False, model_name=model_name)
    for model_name in (
        "local_fasttext",
        "word2vec-google-news-300",
    )
}

In [ ]:
assert len({frozenset(encoder.model.key_to_index.keys()) for encoder in ENCODERS.values()}) == 1

In [ ]:
import random
from pytorch_lightning import seed_everything

seed_everything(4)

NUM_ANCHORS = 300
NUM_TARGETS = 200
NUM_CLUSTERS = 3
WORDS = sorted(ENCODERS["local_fasttext"].model.key_to_index.keys())
WORDS = [word for word in WORDS if word.isalpha() and len(word) >= 4]
TARGET_WORDS = ["school", "ferrari", "water", "martial"]  # words to take the neighborhoods from
# TARGET_WORDS = random.sample(WORDS, NUM_CLUSTERS)
print(f"{TARGET_WORDS=}")
word2index = {word: i for i, word in enumerate(WORDS)}
TARGETS = torch.zeros(len(WORDS), device="cpu")
target_cluster = [
    [word for word, sim in ENCODERS["local_fasttext"].model.most_similar(target_word, topn=NUM_TARGETS)]
    for target_word in TARGET_WORDS
]

valid_words, valid_targets = [], []
for i, target_cluster in enumerate(target_cluster):
    valid_words.append(TARGET_WORDS[i])
    valid_targets.append(i + 1)
    for word in target_cluster:
        if word in word2index:
            valid_words.append(word)
            valid_targets.append(i + 1)

WORDS = valid_words
TARGETS = valid_targets

ANCHOR_WORDS = sorted(random.sample(WORDS, NUM_ANCHORS))  # TODO: stratified

ANCHOR_WORDS[:10]

In [ ]:
from sklearn.decomposition import PCA


def latents_distance(latents):
    assert len(latents) == 2
    for x in latents:
        assert x.shape[1] == 300

    dist = F.pairwise_distance(latents[0], latents[1], p=2).mean()
    return f"{dist:.2f}"


def get_latents(words, encoder: GensimEncoder):
    latents = torch.tensor([encoder.model.get_vector(word) for word in words], device=DEVICE)
    return latents


def to_df(latents, fit_pca: bool = True):
    if fit_pca:
        latents2d = PCA(n_components=2).fit_transform(latents.cpu())
    else:
        latents2d = latents[:, [0, 1]]
    df = pd.DataFrame(
        {
            "x": latents2d[:, 0].tolist(),
            "y": latents2d[:, 1].tolist(),
            "target": TARGETS,
        }
    )
    return df

# Plot stuff

In [ ]:
def plot_bg(
    ax,
    df,
    cmap,
    norm,
    size=0.5,
    bg_alpha=0.01,
):
    """Create and return a plot of all our movie embeddings with very low opacity.
    (Intended to be used as a basis for further - more prominent - plotting of a
    subset of movies. Having the overall shape of the map space in the background is
    useful for context.)
    """
    ax.scatter(df.x, df.y, c=cmap(norm(df["target"])), alpha=bg_alpha, s=size)
    return ax


def hightlight_cluster(
    ax,
    df,
    target,
    alpha,
    cmap,
    norm,
    size=0.5,
):
    cluster_df = df[df["target"] == target]
    ax.scatter(cluster_df.x, cluster_df.y, c=cmap(norm(cluster_df["target"])), alpha=alpha, s=size)


def plot_latent_space(ax, df, targets, size, cmap, norm, bg_alpha=0.1, alpha=0.5):
    ax = plot_bg(ax, df, bg_alpha=bg_alpha, cmap=cmap, norm=norm)
    for target in targets:
        hightlight_cluster(ax, df, target, alpha=alpha, size=size, cmap=cmap, norm=norm)
    return ax

## AE

In [ ]:
ae_latents = {}
anchors_latents = {}
for enc_name, encoder in ENCODERS.items():
    ae_latents[enc_name] = get_latents(words=WORDS, encoder=encoder)
    anchors_latents[enc_name] = get_latents(words=ANCHOR_WORDS, encoder=encoder)

import copy

original_ae_latents = copy.deepcopy(ae_latents)
original_anchor_latents = copy.deepcopy(anchors_latents)

## Rel Attention Quantized

In [ ]:
from sklearn.decomposition import PCA
from rae.modules.attention import *


col_config = (
    (None, None),
    #     (SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.0001),
    #     (SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.05),
    #     (SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.1),
    #     (SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.3),
    ("cluster", 1),
    ("cluster", 1.5),
    ("cluster", 2),
    # ('kmeans', 3),
    #     (SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.6),
    #     (SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.7),
    #     (SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.8),
    #    (SimilaritiesQuantizationMode.DIFFERENTIABLE_ROUND, 0.9),
)
N_ROWS = len(ENCODERS)
N_COLS = len(col_config) + 1

plt.rcParams.update(bundles.icml2022())
plt.rcParams.update(figsizes.icml2022_full(ncols=N_COLS, nrows=N_ROWS, height_to_width_ratio=1.0))

import matplotlib as mpl

num_colors = len(TARGET_WORDS)
cmap = mpl.colors.ListedColormap(plt.cm.get_cmap("Set1", 10).colors[:num_colors], name="rgb", N=num_colors)
# cmap = plt.cm.get_cmap("Set1", 5)
norm = plt.Normalize(min(TARGETS), max(TARGETS))

fig, axes = plt.subplots(dpi=150, nrows=N_ROWS, ncols=N_COLS, sharey=True, sharex=True, squeeze=True)


TARGETS_HIGHTLIGHT = [1]
for ax_encoders, (_, latents) in zip(axes, ae_latents.items()):

    plot_latent_space(
        ax_encoders[0],
        to_df(latents),
        targets=TARGETS_HIGHTLIGHT,
        size=0.75,
        bg_alpha=0.25,
        alpha=1,
        cmap=cmap,
        norm=norm,
    )

distances = {"absolute": latents_distance(list(ae_latents.values()))}

for col_i, (quant_mode, bin_size) in enumerate(col_config):
    rel_attention = RelativeAttention(
        n_anchors=NUM_ANCHORS,
        n_classes=len(set(TARGETS)),
        similarity_mode=RelativeEmbeddingMethod.INNER,
        values_mode=ValuesMethod.SIMILARITIES,
        normalization_mode=NormalizationMode.L2,
        #  output_normalization_mode=OutputNormalization.L2,
        #  similarities_quantization_mode=quant_mode,
        #  similarities_bin_size=bin_size,
        #  similarities_num_clusters=bin_size,
        absolute_quantization_mode=quant_mode,
        absolute_bin_size=bin_size,
        absolute_num_clusters=bin_size,
    )
    assert sum(x.numel() for x in rel_attention.parameters()) == 0
    rels = []
    for row_axes, (enc_name, latents), (a_enc_name, a_latents) in zip(
        axes, ae_latents.items(), anchors_latents.items()
    ):
        assert enc_name == a_enc_name
        rel = rel_attention(x=latents, anchors=a_latents)[AttentionOutput.SIMILARITIES]
        rels.append(rel)
        plot_latent_space(
            row_axes[col_i + 1],
            to_df(rel),
            targets=TARGETS_HIGHTLIGHT,
            size=0.75,
            bg_alpha=0.25,
            alpha=1,
            cmap=cmap,
            norm=norm,
        )
    distances[f"relative({quant_mode}, {bin_size})"] = latents_distance(rels)

distances

In [ ]:
fig.savefig("word-embeddings-spaces.svg", bbox_inches="tight")
!rsvg-convert -f pdf -o 'word-embeddings-spaces.pdf' 'word-embeddings-spaces.svg'
!rm 'word-embeddings-spaces'.svg

# Faiss Search

In [ ]:
from rae.openfaiss import FaissIndex

faiss_indices = {
    enc_name: {model_type: FaissIndex(d=300) for model_type in ["absolute", "relative"]} for enc_name in ENCODERS
}

for enc_name, encoder in ENCODERS.items():
    latents = encoder.model.get_normed_vectors()
    faiss_indices[enc_name]["absolute"].add_vectors(
        embeddings=list(zip(encoder.model.key_to_index.keys(), latents)), normalize=False
    )

    rel_attention = RelativeAttention(
        n_anchors=NUM_ANCHORS,
        n_classes=len(set(TARGETS)),
        similarity_mode=RelativeEmbeddingMethod.INNER,
        values_mode=ValuesMethod.SIMILARITIES,
        normalization_mode=NormalizationMode.L2,
        #  output_normalization_mode=OutputNormalization.L2,
        #  similarities_quantization_mode=quant_mode,
        #  similarities_bin_size=bin_size,
        #  similarities_num_clusters=bin_size,
        absolute_quantization_mode="cluster",
        absolute_bin_size=1.5,  # ignored
        absolute_num_clusters=1.5,
    )
    latents = torch.as_tensor(latents, dtype=torch.float32)
    relative_representation = rel_attention(x=latents, anchors=anchors_latents[enc_name])
    print(relative_representation)

    print(enc_name, encoder)

In [ ]:
zip(encoder.model.key_to_index.keys(), encoder.model.get_normed_vectors())

# Optimal transofrm

In [ ]:
from rae.modules.attention import *

ae, _ = parse_checkpoint(
    module_class=PL_MODULE,
    checkpoint_path=checkpoints["mnist"]["ae"][0],
    map_location="cpu",
)

att = RelativeAttention(
    n_anchors=anchors_batch.shape,
    n_classes=len(set(targets)),
    similarity_mode=RelativeEmbeddingMethod.INNER,
    values_mode=ValuesMethod.SIMILARITIES,
    normalization_mode=NormalizationMode.L2,
    output_normalization_mode=OutputNormalization.NONE,
    similarities_quantization_mode=None,
    similarities_bin_size=None,
    # absolute_quantization_mode=AbsoluteQuantizationMode.DIFFERENTIABLE_ROUND,
    # absolute_bin_size=bin_size
)
att_q = RelativeAttention(
    n_anchors=anchors_batch.shape,
    n_classes=len(set(targets)),
    similarity_mode=RelativeEmbeddingMethod.INNER,
    values_mode=ValuesMethod.SIMILARITIES,
    normalization_mode=NormalizationMode.L2,
    output_normalization_mode=OutputNormalization.NONE,
    similarities_quantization_mode=SimilaritiesQuantizationMode.CUSTOM_ROUND,
    similarities_bin_size=0.1,
    # absolute_quantization_mode=AbsoluteQuantizationMode.DIFFERENTIABLE_ROUND,
    # absolute_bin_size=0.1
)

ae.eval()
images_z = ae(images_batch)[Output.DEFAULT_LATENT].detach()
anchors_z = ae(anchors_batch)[Output.DEFAULT_LATENT].detach()

In [ ]:
from tqdm import tqdm
import torch
from torch.optim.adam import Adam

# Absolut
from scipy.stats import ortho_group

opt_isometry = torch.tensor(ortho_group.rvs(images_z.shape[-1]), dtype=torch.float, requires_grad=True)
opt_shift = torch.zeros(images_z.shape[-1], dtype=torch.float, requires_grad=True)

opt = Adam([opt_isometry, opt_shift], lr=1e-4)


def transform(x):
    return x @ opt_isometry + opt_shift


R = 1000
Q = 1
I = 1000
S = 0
for i in (bar := tqdm(range(100))):
    rel = att(x=images_z, anchors=anchors_z)[AttentionOutput.SIMILARITIES]
    rel_iso = att(x=transform(images_z), anchors=transform(anchors_z))[AttentionOutput.SIMILARITIES]
    rel_dist = F.mse_loss(rel, rel_iso, reduction="sum")
    rel_loss = -rel_dist * R

    qrel = att_q(x=images_z, anchors=anchors_z)[AttentionOutput.SIMILARITIES]
    qrel_iso = att_q(x=transform(images_z), anchors=transform(anchors_z))[AttentionOutput.SIMILARITIES]
    qrel_dist = F.mse_loss(qrel, rel_iso, reduction="sum")
    qrel_loss = qrel_dist * Q

    t_temp = opt_isometry @ opt_isometry.T
    iso_loss = ((t_temp - t_temp.diag().diag()) ** 2).sum() * I
    # iso_loss = (t_temp ** 2 - torch.eye(t_temp.shape[0])).sum() * I
    shift_loss = opt_shift.abs().sum() * S
    loss = rel_loss + qrel_loss + iso_loss + shift_loss

    bar.set_description(f"Rel: {rel_loss.item():3f} \t Qua: {qrel_loss.item():3f} \t  Iso: {iso_loss.item():3f}")
    loss.backward()
    opt.step()
    opt.zero_grad()

rel = att(x=images_z, anchors=anchors_z)[AttentionOutput.SIMILARITIES]
rel_iso = att(x=transform(images_z), anchors=transform(anchors_z))[AttentionOutput.SIMILARITIES]
print("Relative mse:", F.mse_loss(rel, rel_iso, reduction="sum"))

qrel = att_q(x=images_z, anchors=anchors_z)[AttentionOutput.SIMILARITIES]
qrel_iso = att_q(x=transform(images_z), anchors=transform(anchors_z))[AttentionOutput.SIMILARITIES]
print("Quantized mse:", F.mse_loss(qrel, qrel_iso, reduction="sum"))

In [ ]:
ae.eval()
images_z = ae(images_batch)[Output.DEFAULT_LATENT].detach()
anchors_z = ae(anchors_batch)[Output.DEFAULT_LATENT].detach()

rel = att(x=images_z, anchors=anchors_z)[AttentionOutput.SIMILARITIES]
rel_iso = att(x=transform(images_z), anchors=transform(anchors_z))[AttentionOutput.SIMILARITIES]
print("Relative mse:", F.mse_loss(rel, rel_iso, reduction="sum"))

qrel = att_q(x=images_z, anchors=anchors_z)[AttentionOutput.SIMILARITIES]
qrel_iso = att_q(x=transform(images_z), anchors=transform(anchors_z))[AttentionOutput.SIMILARITIES]
print("Quantized mse:", F.mse_loss(qrel, qrel_iso, reduction="sum"))

In [ ]:
qrel

In [ ]:
b = torch.as_tensor(0.5)
x = torch.linspace(-1, 1, 200)
y = x - torch.sin(2 * torch.pi * x) / (2 * torch.pi)

a = 1
f = 1 / b
s = 0
y = x - a * torch.cos(2 * torch.pi * f * x + s) / (2 * torch.pi * f)

fig, ax = plt.subplots(1, 1, dpi=150)
f = ax.plot(
    x,
    y,
)